In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
# import tensorflow_hub as hub
# import tensorflow_text
from tqdm import tqdm

In [ ]:
df=pd.read_csv('../input/nlp-getting-started/train.csv')
test_df=pd.read_csv('../input/nlp-getting-started/test.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(['keyword','location','id'],axis='columns',inplace=True)

In [ ]:
sns.countplot(df['target'], palette="plasma")
fig = plt.gcf()
fig.set_size_inches(5,5)
plt.title('target')

In [ ]:
df.shape

In [ ]:
df.head()

## Text Preprocessing

In [ ]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
     "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited","ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [ ]:
def text_cleaner(text):
    temp = text.lower()
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    temp = url_pattern.sub(r'', temp)
    html=re.compile(r'<[^>]+>')
    temp = html.sub(r'',temp)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    temp = emoji_pattern.sub(r'', temp)
    
    table=str.maketrans('','',string.punctuation)
    temp = temp.translate(table)
    
    tem=[]
    for word in temp.split():
        if word in abbreviations.keys():
            tem.append(abbreviations[word])
        else:
            tem.append(word)
    temp=tem
    
    stop_words = set(stopwords.words('english'))
    le=WordNetLemmatizer()
    tokens = [le.lemmatize(word) for word in temp if not word in stop_words]
    
    long_words=[]
    for i in tokens:
        if len(i)>1:                                            
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [ ]:
df['text'] = df['text'].apply(lambda text: text_cleaner(text))
df.head()

#### Drop empty texts

In [ ]:
df.replace('', np.nan, inplace=True)
df.dropna(axis='rows',inplace=True)

In [ ]:
def leng(text):
    return len(text.split())

In [ ]:
df['text_length']=df['text'].apply(lambda text: leng(text))
plt.boxplot(df['text_length'])
plt.show()

In [ ]:
maximum = df['text_length'].max()

## Relabeling mislabeled tweets

In [ ]:
df_mislabeled = df.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
index_mislabeled = df_mislabeled.index.tolist()

length = len(index_mislabeled)
print(length)

There are 73 texts that are mislabeled.

In [ ]:
df_mistext_check_before = df[df['text'].isin(index_mislabeled)].sort_values(by = 'text')
df_mistext_check_before.head(10)

In [ ]:
majority_df = df_mistext_check_before.groupby(['text'])['target'].mean()#apply(lambda x: x.mode())

In [ ]:
majority_df

We will relabel them according to mean of the labels of the same tweet.

In [ ]:
def relabel(row, majority_index):
    ind = ''
    if row['text'] in majority_index:
        ind = row['text']
        if majority_df[ind] < 0.5:
            return 0
        else:
            return 1
    else:
        return row['target']


In [ ]:
# df['target'] = df[df['text'].isin(majority_df.index.tolist())].apply(lambda row['target']: majority_df[row['text']]) 
df['target'] = df.apply( lambda row: relabel(row, majority_df.index), axis = 1)
# df['target']=df['target'].apply(lambda row : int(row))

In [ ]:
df_mislabeled = df.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
index_mislabeled = df_mislabeled.index.tolist()

length = len(index_mislabeled)
print(length)

After relabeling, we can see that there are no mislabeled tweets.

In [ ]:
df.info()

## Glove Vector Embeddings

In [ ]:
x_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>') 
x_tokenizer.fit_on_texts(list(df['text']))

In [ ]:
df['sequences']=x_tokenizer.texts_to_sequences(df['text'])

In [ ]:
df['sequences'].iloc[0]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid=train_test_split(df['sequences'],df['target'].values,test_size=0.2,shuffle=True) 

In [ ]:
x_train   =   tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=maximum, padding='post')
x_valid   =   tf.keras.preprocessing.sequence.pad_sequences(x_valid,maxlen=maximum, padding='post')

x_voc   =  len(x_tokenizer.word_index) + 1 

In [ ]:
x_word_index = x_tokenizer.word_index

In [ ]:
len(x_word_index)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

In [ ]:
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((x_voc+1, 100));
for word, i in x_word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [ ]:
embeddings_matrix.shape

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(x_voc+1, 100, input_length=maximum, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences = False)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

num_epochs = 20

history = model.fit(x_train, y_train, epochs=num_epochs, validation_data=(x_valid, y_valid), verbose=2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'])
plt.show()

In [ ]:
test_df.drop(['location','id','keyword'],axis='columns',inplace=True)

In [ ]:
test_df.head()

In [ ]:
test_df['text'] = test_df['text'].apply(lambda text: text_cleaner(text))

In [ ]:
test_df.head()

In [ ]:
test_seq = x_tokenizer.texts_to_sequences(test_df['text'])
test_seq = tf.keras.preprocessing.sequence.pad_sequences(test_seq,maxlen=maximum, padding='post')

In [ ]:
preds=model.predict(test_seq)
preds=[int(round(x[0])) for x in preds]
samp=pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
samp["target"]=preds
samp["target"].value_counts()

In [ ]:
samp.to_csv("outputGlove.csv",index=False)

In [ ]:
print(test_df.iloc[-15])

In [ ]:
samp['target'].iloc[-15]